#### Gathering all the prediction results

In [141]:
# Assigning clusters to the dataset

kmeans = KMeans(n_clusters= 3 ,**kmeans_kwargs)
kmeans.fit(X_train_scaled)
k_predictions = kmeans.predict(X_test_scaled)
k_predictions_all = kmeans.predict(features_scaled)

In [142]:
# Defining a dictionary with the best ML models

models = {'dtc':[predictions_dtc_all,predictions_prob_dtc_all,dtc], 'rfc':[predictions_rfc_all, predictions_prob_rfc_all,rfc], 'lr':[predictions_lr_all,predictions_prob_lr_all,lr],'lda':[predictions_lda_all,predictions_prob_lda_all,lda],'svc':[predictions_svc_all, predictions_prob_svc_all,svc]}

# Applying these models to the whole dataset

final_dataset_fin = pd.DataFrame()
    
for model in models:
    
    final_dataset_aux = historical_data.copy()
    final_dataset_aux['cluster'] = k_predictions_all
    final_dataset_aux['Prob_Home%'] = np.round(models[model][1][:,2]*100,2)
    final_dataset_aux['Prob_Draw%'] = np.round(models[model][1][:,1]*100,2)
    final_dataset_aux['Prob_Away%'] = np.round(models[model][1][:,0]*100,2)
    final_dataset_aux['Prediction'] = models[model][0]
    final_dataset_aux['Prediction'] = lab_enc_results.inverse_transform(final_dataset_aux['Prediction'])
    final_dataset_aux['hit_result'] = (final_dataset_aux['FTR'] == final_dataset_aux['Prediction'])*1
    final_dataset_aux['paid_quota'] = ((final_dataset_aux['FTR'] == 'A') * final_dataset_aux['B365A']) + ((final_dataset_aux['FTR'] == 'D') * final_dataset_aux['B365D']) + ((final_dataset_aux['FTR'] == 'H') * final_dataset_aux['B365H'])
    final_dataset_aux['Prize'] = final_dataset_aux['hit_result'] * final_dataset_aux['paid_quota']
    final_dataset_aux['Profit'] =  final_dataset_aux['Prize'] - 1
    final_dataset_aux['Difficulty'] = final_dataset_aux['paid_quota'].astype('int')
    final_dataset_aux['Model'] = model
    final_dataset_fin = final_dataset_fin.append(final_dataset_aux)

In [143]:
# Exporting results

final_dataset_fin.to_excel('user_report/final_dataset_fin.xlsx')

In [144]:
final_dataset_fin.tail()

,Div,Date,season,HomeTeam,AwayTeam,match_home,match_away,B365H,B365D,B365A,...,Prob_Home%,Prob_Draw%,Prob_Away%,Prediction,hit_result,paid_quota,Prize,Profit,Difficulty,Model
4435,SP1,2021-05-16,2021,Cadiz,Elche,37,37,3.2,3.1,2.35,...,55.33,26.48,18.19,H,0,2.35,0,-1,2,svc
4436,SP1,2021-05-16,2021,Getafe,Levante,37,37,1.61,3.8,5.75,...,49.57,27.31,23.12,H,1,1.61,1.61,0.61,1,svc
4437,SP1,2021-05-16,2021,Sociedad,Valladolid,37,37,1.61,3.75,6,...,60.57,29.42,10.01,H,1,1.61,1.61,0.61,1,svc
4438,SP1,2021-05-16,2021,Valencia,Eibar,37,37,2.87,3.4,2.37,...,33.93,29.25,36.82,H,1,2.87,2.87,1.87,2,svc
4439,SP1,2021-05-16,2021,Villarreal,Sevilla,37,37,2.37,3.4,2.87,...,37.66,28.18,34.16,H,1,2.37,2.37,1.37,2,svc
